In [ ]:
#Run every cell, in order, until instructed otherwise

In [104]:
import pandas as pd
import numpy as npy
from datetime import datetime, date
import matplotlib.pyplot as plt
import random, sklearn, scipy, itertools
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score

In [105]:
#this cell contains all functions defined for building columns in ufcfightscrap
#converts from '%B %d, %Y' (i.e. August 22, 2020) to date (i.e. 2020-08-22)
def convert_to_datetime(day1):
    return datetime.strptime(day1, '%B %d, %Y').date()
convert_to_datetime_vect= npy.vectorize(convert_to_datetime)

#this age function is written in such a stupid way
def age(birthDate,day=date.today(),form1='%B %d, %Y',form2='%B %d, %Y'):
    if birthDate=='--':
        aa='unknown'
    elif type(birthDate)==str and not type(day)==str:
        bd=datetime.strptime(birthDate, form1)
        today = day
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    elif type(birthDate)==str and type(day)==str:
        bd=datetime.strptime(birthDate, form1)
        today = datetime.strptime(day, form2)
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    elif not type(birthDate)==str and type(day)==str:
        bd=birthDate
        today = datetime.strptime(day, form2)
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    else:
        bd=birthDate
        today = day
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    return float(aa)

age_vect= npy.vectorize(age)

def fighter_age(fighter,day=date.today(),form1='%B %d, %Y',form2='%B %d, %Y'):
    a=0
    for i in range(len(ufcfighterscrap['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            dob=datetime.strptime(ufcfighterscrap['dob'][i], '%b %d, %Y').strftime('%B %d, %Y')
            a=age(dob,day,form1,form2)
            break
    return float(a)

def wins_before(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if time_diff(ufcfightscrap['date'][i],day1)>0 and ufcfightscrap['result'][i]=='W':
            summ+=1
        else:
            summ+=0
    return float(summ)

def losses_before(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if time_diff(ufcfightscrap['date'][i],day1)>0 and ufcfightscrap['result'][i]=='L':
            summ+=1
        else:
            summ+=0
    return float(summ)

#functions for height and reach
def fighter_height(fighter):
    a=0
    for i in range(len(ufcfighterscrap['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            a=ufcfighterscrap['height'][i]
            break
    if a=='--':
        b='unknown'
    elif a[4]=='"':
        b=int(a[0])*30.48+int(a[3])*2.54
    else:
        b=int(a[0])*30.48+int(a[3]+a[4])*2.54
    try:
        return float(b)
    except:
        return b

def fighter_reach(fighter):
    a=0
    for i in range(len(ufcfighterscrap['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            a=ufcfighterscrap['reach'][i]
            break
    if a=='--':
        b='unknown'
    else:
        b=int(a[0]+a[1])*2.54
    try:
        return float(b)
    except:
        return b

wins_before_vect= npy.vectorize(wins_before)
losses_before_vect= npy.vectorize(losses_before)
fighter_height_vect= npy.vectorize(fighter_height)
fighter_reach_vect= npy.vectorize(fighter_reach)

#day1 should be input in the form '%B %d, %Y' i.e. 'August 20, 1962'
#conversions can be made via day=datetime.strptime(ufcfightsML_known_df['date'][i], '%B %d, %Y').strftime('%b %d, %Y')
def time_diff(day1,day2=date.today()):
    if day2==date.today():
        answer=(day2-datetime.strptime(day, '%B %d, %Y')).days
    else:
        answer=(datetime.strptime(day2, '%B %d, %Y')-datetime.strptime(day1, '%B %d, %Y')).days
    return answer

#we now vectorize this to use in pandas/numpy
time_diff_vect= npy.vectorize(time_diff)

#can make a single function to do all of these... actually maybe the count function would even work as is

def L5Y_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='W':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return float(summ)

def L5Y_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='L':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return float(summ)

def L2Y_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='W':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return float(summ)

def L2Y_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='L':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return float(summ)

def ko_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1) and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
    return float(summ)

def ko_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1) and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
    return float(summ)

def L5Y_ko_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return float(summ)

def L5Y_ko_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return float(summ)

def L2Y_ko_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return float(summ)

def L2Y_ko_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return float(summ)

def sub_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1) and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
    return float(summ)

def sub_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1) and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
    return float(summ)

def L5Y_sub_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return float(summ)

def L5Y_sub_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return float(summ)

def L2Y_sub_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return float(summ)

def L2Y_sub_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return float(summ)

#vectorize all of these functions
L5Y_wins_vect= npy.vectorize(L5Y_wins)
L5Y_losses_vect= npy.vectorize(L5Y_losses)
L2Y_wins_vect= npy.vectorize(L2Y_wins)
L2Y_losses_vect= npy.vectorize(L2Y_losses)
ko_wins_vect= npy.vectorize(ko_wins)
ko_losses_vect= npy.vectorize(ko_losses)
L5Y_ko_wins_vect= npy.vectorize(L5Y_ko_wins)
L5Y_ko_losses_vect= npy.vectorize(L5Y_ko_losses)
L2Y_ko_wins_vect= npy.vectorize(L2Y_ko_wins)
L2Y_ko_losses_vect= npy.vectorize(L2Y_ko_losses)
sub_wins_vect= npy.vectorize(sub_wins)
sub_losses_vect= npy.vectorize(sub_losses)
L5Y_sub_wins_vect= npy.vectorize(L5Y_sub_wins)
L5Y_sub_losses_vect= npy.vectorize(L5Y_sub_losses)
L2Y_sub_wins_vect= npy.vectorize(L2Y_sub_wins)
L2Y_sub_losses_vect= npy.vectorize(L2Y_sub_losses)

#for columns like fighter_rec which contains the information for the opponent as well, we use the following
def opponent_column(stat):
    col=dict()
    for i in range(len(ufcfightscrap['fighter'])):
        if i%2==0:
            col[i]=ufcfightscrap[stat][i+1]
        else:
            col[i]=ufcfightscrap[stat][i-1]
    statdict={'stat':col}
    return pd.DataFrame (statdict, columns = ['stat'])

#enter date unabbreviated 'July 4, 2019'
#here the average gives avg per fight. Later in avg_count we change to average per time spent in octagon
def count(stat, guy,inf_abs, total_L5Y_L2Y_avg, day1=date.today().strftime('%B %d, %Y')):
    summ=0
    if total_L5Y_L2Y_avg=='total' or total_L5Y_L2Y_avg=='avg':
        good_indices_1=[i for i in ufcfightscrap.index.values if time_diff(ufcfightscrap['date'][i],day1)>0]
    elif total_L5Y_L2Y_avg=='L2Y':
        good_indices_1=[i for i in ufcfightscrap.index.values if 0<time_diff(ufcfightscrap['date'][i],day1)<730]
    else:
        good_indices_1=[i for i in ufcfightscrap.index.values if 0<time_diff(ufcfightscrap['date'][i],day1)<1825]
    if inf_abs=='inf':
        good_indices_2=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    else:
        good_indices_2=[i for i in ufcfightscrap.index.values if ufcfightscrap['opponent'][i]==guy]
    good_indices=[i for i in good_indices_1 if i in good_indices_2]
    if total_L5Y_L2Y_avg!='avg':
        for i in good_indices:
            summ+=ufcfightscrap[stat][i]
    else:
        for i in good_indices:
            summ+=ufcfightscrap[stat][i]
        day1=convert_date_to_abbrev(day1)
        number_fights=wins_before(guy,day1)+losses_before(guy,day1)
        summ=summ/float(number_fights)
    return summ
        
#note a better average is per time not per fight. We will do each stat as an average per one minutes

#enter time in the form 'August 24, 2018'
def time_in_octagon(guy,day1=date.today().strftime('%B %d, %Y')):
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if time_diff(ufcfightscrap['date'][i],day1)>0 and ufcfightscrap['fighter'][i]==guy] 
    for i in good_indices:
        if ufcfightscrap['time'][i][2]==':':
            summ=int(ufcfightscrap['time'][i][0:2])+int(ufcfightscrap['time'][i][3:])/60.0
        else:
            summ+=5*(ufcfightscrap['round'][i]-1)+int(ufcfightscrap['time'][i][0])+int(ufcfightscrap['time'][i][2:])/60.0
    return summ

#enter date unabbreviated 'July 4, 2019'
#gives takedowns per minute
def avg_count(stat, guy,inf_abs, day1=date.today().strftime('%B %d, %Y')):
    summ=0
    if inf_abs=='inf':
        good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy and time_diff(ufcfightscrap['date'][i],day1)>0]
    else:
        good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['opponent'][i]==guy and time_diff(ufcfightscrap['date'][i],day1)>0]
    for i in good_indices:
        summ+=ufcfightscrap[stat][i]
    t= time_in_octagon(guy,day1)
    if t==0:
        summ=0
    else:
        summ=summ/t
    return summ

#vectorize these functions
count_vect= npy.vectorize(count)
avg_count_vect= npy.vectorize(avg_count)

#need to define a model first before calling the following functions
#REDEFINE THIS AFTER GETTING ALL STATS COMPILED. WE CURRENTLY CANNOT PREDICT
#I've redefined this in such a way to predict what happens when fighter1 in their day1 version fights fighter2
#in their day2 version. Meaning we could compare for example 2014 Tyron Woodley to 2019 Colby Covington
def ufc_prediction_tuple(fighter1,fighter2,day1=date.today(),day2=date.today()):
    tupp=[wins_before(fighter1,day1)-wins_before(fighter2,day2),
            losses_before(fighter1,day1)-losses_before(fighter2,day2),
            L5Y_wins(fighter1,day1)-L5Y_wins(fighter2,day2),
            L5Y_losses(fighter1,day1)-L5Y_losses(fighter2,day2),
            L2Y_wins(fighter1,day1)-L2Y_wins(fighter2,day2),
            L2Y_losses(fighter1,day1)-L2Y_losses(fighter2,day2),
            ko_wins(fighter1,day1)-ko_wins(fighter2,day2),
            ko_losses(fighter1,day1)-ko_losses(fighter2,day2),
            L5Y_ko_wins(fighter1,day1)-L5Y_ko_wins(fighter2,day2),
            L5Y_ko_losses(fighter1,day1)-L5Y_ko_losses(fighter2,day2),
            L2Y_ko_wins(fighter1,day1)-L2Y_ko_wins(fighter2,day2),
            L2Y_ko_losses(fighter1,day1)-L2Y_ko_losses(fighter2,day2),
            sub_wins(fighter1,day1)-sub_wins(fighter2,day2),
            sub_losses(fighter1,day1)-sub_losses(fighter2,day2),
            L5Y_sub_wins(fighter1,day1)-L5Y_sub_wins(fighter2,day2),
            L5Y_sub_losses(fighter1,day1)-L5Y_sub_losses(fighter2,day2),
            L2Y_sub_wins(fighter1,day1)-L2Y_sub_wins(fighter2,day2),
            L2Y_sub_losses(fighter1,day1)-L2Y_sub_losses(fighter2,day2),
            fighter_age(fighter1,day1),fighter_age(fighter2,day2),
            fighter_height(fighter1)-fighter_height(fighter2),
            fighter_reach(fighter1)-fighter_reach(fighter2),
            avg_count('knockdowns',fighter1,'inf',day1), avg_count('pass',fighter1,'inf',day1),
            avg_count('reversals',fighter1,'inf',day1),avg_count('sub_attempts',fighter1,'inf',day1),
            avg_count('takedowns_landed',fighter1,'inf',day1),avg_count('takedowns_attempts',fighter1,'inf',day1),
            avg_count('sig_strikes_landed',fighter1,'inf',day1),avg_count('sig_strikes_attempts',fighter1,'inf',day1),
            avg_count('total_strikes_landed',fighter1,'inf',day1),avg_count('total_strikes_attempts',fighter1,'inf',day1),
            avg_count('head_strikes_landed',fighter1,'inf',day1),avg_count('head_strikes_attempts',fighter1,'inf',day1),
            avg_count('body_strikes_landed',fighter1,'inf',day1),avg_count('body_strikes_attempts',fighter1,'inf',day1),
            avg_count('leg_strikes_landed',fighter1,'inf',day1),avg_count('leg_strikes_attempts',fighter1,'inf',day1),
            avg_count('distance_strikes_landed',fighter1,'inf',day1),avg_count('distance_strikes_attempts',fighter1,'inf',day1),
            avg_count('clinch_strikes_landed',fighter1,'inf',day1),avg_count('clinch_strikes_attempts',fighter1,'inf',day1),
            avg_count('ground_strikes_landed',fighter1,'inf',day1),avg_count('ground_strikes_attempts',fighter1,'inf',day1),
            avg_count('knockdowns',fighter1,'abs',day1), avg_count('pass',fighter1,'abs',day1),
            avg_count('reversals',fighter1,'abs',day1),avg_count('sub_attempts',fighter1,'abs',day1),
            avg_count('takedowns_landed',fighter1,'abs',day1),avg_count('takedowns_attempts',fighter1,'abs',day1),
            avg_count('sig_strikes_landed',fighter1,'abs',day1),avg_count('sig_strikes_attempts',fighter1,'abs',day1),
            avg_count('total_strikes_landed',fighter1,'abs',day1),avg_count('total_strikes_attempts',fighter1,'abs',day1),
            avg_count('head_strikes_landed',fighter1,'abs',day1),avg_count('head_strikes_attempts',fighter1,'abs',day1),
            avg_count('body_strikes_landed',fighter1,'abs',day1),avg_count('body_strikes_attempts',fighter1,'abs',day1),
            avg_count('leg_strikes_landed',fighter1,'abs',day1),avg_count('leg_strikes_attempts',fighter1,'abs',day1),
            avg_count('distance_strikes_landed',fighter1,'abs',day1),avg_count('distance_strikes_attempts',fighter1,'abs',day1),
            avg_count('clinch_strikes_landed',fighter1,'abs',day1),avg_count('clinch_strikes_attempts',fighter1,'abs',day1),
            avg_count('ground_strikes_landed',fighter1,'abs',day1),avg_count('ground_strikes_attempts',fighter1,'abs',day1),
            avg_count('knockdowns',fighter2,'inf',day2), avg_count('pass',fighter2,'inf',day2),
            avg_count('reversals',fighter2,'inf',day2),avg_count('sub_attempts',fighter2,'inf',day2),
            avg_count('takedowns_landed',fighter2,'inf',day2),avg_count('takedowns_attempts',fighter2,'inf',day2),
            avg_count('sig_strikes_landed',fighter2,'inf',day2),avg_count('sig_strikes_attempts',fighter2,'inf',day2),
            avg_count('total_strikes_landed',fighter2,'inf',day2),avg_count('total_strikes_attempts',fighter2,'inf',day2),
            avg_count('head_strikes_landed',fighter2,'inf',day2),avg_count('head_strikes_attempts',fighter2,'inf',day2),
            avg_count('body_strikes_landed',fighter2,'inf',day2),avg_count('body_strikes_attempts',fighter2,'inf',day2),
            avg_count('leg_strikes_landed',fighter2,'inf',day2),avg_count('leg_strikes_attempts',fighter2,'inf',day2),
            avg_count('distance_strikes_landed',fighter2,'inf',day2),avg_count('distance_strikes_attempts',fighter2,'inf',day2),
            avg_count('clinch_strikes_landed',fighter2,'inf',day2),avg_count('clinch_strikes_attempts',fighter2,'inf',day2),
            avg_count('ground_strikes_landed',fighter2,'inf',day2),avg_count('ground_strikes_attempts',fighter2,'inf',day2),
            avg_count('knockdowns',fighter2,'abs',day2), avg_count('pass',fighter2,'abs',day2),
            avg_count('reversals',fighter2,'abs',day2),avg_count('sub_attempts',fighter2,'abs',day2),
            avg_count('takedowns_landed',fighter2,'abs',day2),avg_count('takedowns_attempts',fighter2,'abs',day2),
            avg_count('sig_strikes_landed',fighter2,'abs',day2),avg_count('sig_strikes_attempts',fighter2,'abs',day2),
            avg_count('total_strikes_landed',fighter2,'abs',day2),avg_count('total_strikes_attempts',fighter2,'abs',day2),
            avg_count('head_strikes_landed',fighter2,'abs',day2),avg_count('head_strikes_attempts',fighter2,'abs',day2),
            avg_count('body_strikes_landed',fighter2,'abs',day2),avg_count('body_strikes_attempts',fighter2,'abs',day2),
            avg_count('leg_strikes_landed',fighter2,'abs',day2),avg_count('leg_strikes_attempts',fighter2,'abs',day2),
            avg_count('distance_strikes_landed',fighter2,'abs',day2),avg_count('distance_strikes_attempts',fighter2,'abs',day2),
            avg_count('clinch_strikes_landed',fighter2,'abs',day2),avg_count('clinch_strikes_attempts',fighter2,'abs',day2),
            avg_count('ground_strikes_landed',fighter2,'abs',day2),avg_count('ground_strikes_attempts',fighter2,'abs',day2),
            stance(fighter1),stance(fighter2)]
    for i in range(len(tupp)):
        tupp[i]=float(tupp[i])
    return tupp

def mlp_prediction_tuple(fighter1,fighter2,day1=date.today(),day2=date.today()):
    tupp=[ko_wins(fighter1,day1),
            ko_losses(fighter1,day1),
            ko_wins(fighter2,day2),
            ko_losses(fighter2,day2),
            L5Y_ko_wins(fighter1,day1),
            L5Y_ko_losses(fighter1,day1),
            L2Y_ko_wins(fighter1,day1),
            L2Y_ko_losses(fighter1,day1),
            L5Y_ko_wins(fighter2,day2),
            L5Y_ko_losses(fighter2,day2),
            L2Y_ko_wins(fighter2,day2),
            L2Y_ko_losses(fighter2,day2),
            sub_wins(fighter1,day1),
            sub_losses(fighter1,day1),
            sub_wins(fighter2,day2),
            sub_losses(fighter2,day2),
            avg_count('head_strikes_landed',fighter1,'inf',day1),avg_count('head_strikes_attempts',fighter1,'inf',day1),
            avg_count('head_strikes_landed',fighter1,'abs',day1),avg_count('head_strikes_attempts',fighter1,'abs',day1),
            avg_count('head_strikes_landed',fighter2,'inf',day2),avg_count('head_strikes_attempts',fighter2,'inf',day2),
            avg_count('head_strikes_landed',fighter2,'abs',day2),avg_count('head_strikes_attempts',fighter2,'abs',day2),
            avg_count('clinch_strikes_landed',fighter1,'inf',day1),avg_count('clinch_strikes_attempts',fighter1,'inf',day1),
            avg_count('clinch_strikes_landed',fighter1,'abs',day1),avg_count('clinch_strikes_attempts',fighter1,'abs',day1),
            avg_count('clinch_strikes_landed',fighter2,'inf',day2),avg_count('clinch_strikes_attempts',fighter2,'inf',day2),
            avg_count('clinch_strikes_landed',fighter2,'abs',day2),avg_count('clinch_strikes_attempts',fighter2,'abs',day2),
            L5Y_sub_wins(fighter1,day1),
            L5Y_sub_losses(fighter1,day1),
            L2Y_sub_wins(fighter1,day1),
            L2Y_sub_losses(fighter1,day1),
            L5Y_sub_wins(fighter2,day2),
            L5Y_sub_losses(fighter2,day2),
            L2Y_sub_wins(fighter2,day2),
            L2Y_sub_losses(fighter2,day2),
            avg_count('body_strikes_landed',fighter1,'inf',day1),
            avg_count('body_strikes_landed',fighter2,'inf',day2)]
    for i in range(len(tupp)):
        tupp[i]=float(tupp[i])
    return tupp
            
            
def model_prediction_tuple(fighter1,fighter2,day1=date.today(),day2=date.today()):
    tupp=[wins_before(fighter1,day1)-wins_before(fighter2,day2),
            losses_before(fighter1,day1)-losses_before(fighter2,day2),
            L5Y_wins(fighter1,day1)-L5Y_wins(fighter2,day2),
            L5Y_losses(fighter1,day1)-L5Y_losses(fighter2,day2),
            L2Y_wins(fighter1,day1)-L2Y_wins(fighter2,day2),
            L2Y_losses(fighter1,day1)-L2Y_losses(fighter2,day2),
            L5Y_ko_wins(fighter1,day1)-L5Y_ko_wins(fighter2,day2),
            L5Y_ko_losses(fighter1,day1)-L5Y_ko_losses(fighter2,day2),
            L2Y_ko_wins(fighter1,day1)-L2Y_ko_wins(fighter2,day2),
            L2Y_ko_losses(fighter1,day1)-L2Y_ko_losses(fighter2,day2),
            L2Y_sub_losses(fighter1,day1)-L2Y_sub_losses(fighter2,day2),
            L5Y_sub_wins(fighter1,day1)-L5Y_sub_wins(fighter2,day2),
            L5Y_sub_losses(fighter1,day1)-L5Y_sub_losses(fighter2,day2),
            L2Y_sub_wins(fighter1,day1)-L2Y_sub_wins(fighter2,day2),
            L2Y_sub_losses(fighter1,day1)-L2Y_sub_losses(fighter2,day2),
            avg_count('reversals',fighter1,'inf',day1),
            avg_count('reversals',fighter1,'abs',day1),
            avg_count('reversals',fighter2,'inf',day2),
            avg_count('reversals',fighter2,'abs',day2),
            avg_count('takedowns_attempts',fighter1,'inf',day1),
            avg_count('takedowns_attempts',fighter1,'abs',day1),
            avg_count('takedowns_attempts',fighter2,'inf',day2),
            avg_count('takedowns_attempts',fighter2,'abs',day2),
            ko_wins(fighter1,day1)-ko_wins(fighter2,day2),
            fighter_age(fighter1,day1),fighter_age(fighter2,day2),
            sub_wins(fighter1,day1)-sub_wins(fighter2,day2),
            ko_losses(fighter1,day1)-ko_losses(fighter2,day2),
            avg_count('body_strikes_landed',fighter1,'inf',day1),
            avg_count('body_strikes_attempts',fighter1,'inf',day1),
            avg_count('body_strikes_landed',fighter1,'abs',day1),
            avg_count('body_strikes_attempts',fighter1,'abs',day1),
            avg_count('body_strikes_landed',fighter2,'inf',day2),
            avg_count('body_strikes_attempts',fighter2,'inf',day2),
            avg_count('body_strikes_landed',fighter2,'abs',day2),
            avg_count('body_strikes_attempts',fighter2,'abs',day2),
            avg_count('takedowns_landed',fighter1,'inf',day1),
            avg_count('takedowns_landed',fighter1,'abs',day1),
            avg_count('takedowns_landed',fighter2,'inf',day2),
            avg_count('takedowns_landed',fighter2,'abs',day2),
            avg_count('head_strikes_attempts',fighter1,'inf',day1),
            avg_count('head_strikes_attempts',fighter1,'abs',day1),
            avg_count('head_strikes_attempts',fighter2,'inf',day2),
            avg_count('head_strikes_attempts',fighter2,'abs',day2),
            avg_count('distance_strikes_landed',fighter1,'inf',day1),
            avg_count('distance_strikes_landed',fighter1,'abs',day1),
            avg_count('distance_strikes_landed',fighter2,'inf',day2),
            avg_count('distance_strikes_landed',fighter2,'abs',day2),
            avg_count('distance_strikes_attempts',fighter1,'inf',day1),
            avg_count('distance_strikes_attempts',fighter1,'abs',day1),
            avg_count('distance_strikes_attempts',fighter2,'inf',day2),
            avg_count('distance_strikes_attempts',fighter2,'abs',day2)]
    for i in range(len(tupp)):
        tupp[i]=float(tupp[i])
    return tupp

#date entered in the form 'Aug 20, 2013'
#predicts what happens if the day1 version of fighter1 fights fighter2 at their day2 version
#by default day1 and day2 are set to today if left unspecified
def ufc_predict(fighter1,fighter2,day1=date.today(), day2=date.today()):
    fighters=[fighter1,fighter2]
    random_guy=fighters[random.choice([0,1])]
    fighter1_fights=(ufcfightscrap['fighter']==fighter1).sum()
    fighter2_fights=(ufcfightscrap['fighter']==fighter2).sum()
    if fighter1_fights==0 and not fighter2_fights==0:
        print('Either you spelled something wrong, or this is '+fighter1+"'s UFC debut... so obviously I do not have enough data to make an educated pick. But since you are DEMANDING an outcome, hmmm... I guess I'll go with "+random_guy+'.')
    elif fighter2_fights==0 and not fighter1_fights==0:
        print('Either you spelled something wrong, or this is '+fighter2+"'s UFC debut... so obviously I do not have enough data to make an educated pick. But since you are DEMANDING an outcome, hmmm... I guess I'll go with "+random_guy+'.') 
    elif fighter1_fights==0 and fighter2_fights==0:
        print('Either you spelled some things wrong, or this is both '+fighter1+"'s "+'and '+fighter2+"'s "+"UFC debuts... so obviously I do not have enough data to make an educated pick. But since you are DEMANDING an outcome, hmmm... I guess I'll go with "+random_guy+'.')
    else:
        model_tuple=model_prediction_tuple(fighter1,fighter2,day1,day2)
        mlp_tuple=mlp_prediction_tuple(fighter1,fighter2,day1,day2)
        value=model.predict([model_tuple])[0]
        result=mlp.predict([mlp_tuple])[0]
        if value=='W':
            print(fighter1+' wins by '+result)
        elif value=='L':
            print(fighter2+' wins by '+result)
        else:
            print("Something has gone wrong")

def stance(fighter):
    a=0
    for i in range(len(ufcfighterscrap['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            a=ufcfighterscrap['stance'][i]
            break
    if a=='Orthodox':
        return 0
    elif a=='Switch':
        return 1
    elif a=='Southpaw':
        return 2
    elif a=='Open Stance':
        return 3
    elif a=='Sideways':
        return 4
    else:
        return 5
    
stance_vect= npy.vectorize(stance)

def subsets(n):
    lists=[]
    for i in range(1,n+1):
        lists.append(i)
    subsets=[[]]
    n=len(lists)
    for i in range(1,n+1):
        data = itertools.combinations(lists,i)
        subsets.extend(list(data))
    return subsets

In [106]:
ufc_fights= pd.read_csv('ufc_fights.csv')
ufc_fights_crap= pd.read_csv('ufc_fights_crap.csv')
ufcfightscrap= pd.read_csv('ufc_fights_crap.csv')
ufcfighterscrap=pd.read_csv('fighter_stats.csv',sep=',')

/opt/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (73,74) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [107]:
#adding mask to get rid of unknown stance
stance_mask_a=ufc_fights['fighter_stance']!=5
stance_mask_b=ufc_fights['opponent_stance']!=5

#gets rid of all unknown, NaN in ufc_fights_stance
mask=ufc_fights['fighter_wins'] != 'unknown'
for col in ufc_fights.columns:
    mask1=ufc_fights[col] != 'unknown'
    mask2=ufc_fights[col] != 'NaN'
    mask3=ufc_fights[col] != '--'
    mask=mask&mask1&mask2&mask3

#gets rid of ufc debut in ufc_fights_stance
maska=ufc_fights['fighter_inf_sig_strikes_attempts_avg'] != 0
maskb=ufc_fights['opponent_inf_sig_strikes_attempts_avg'] != 0

#Getting rid of all fights whose method is DQ, Overturned, or CNC or any draws
method_mask1=ufc_fights['method']!='DQ'
method_mask2=ufc_fights['method']!='Overturned'
method_mask3=ufc_fights['method']!='CNC'
method_mask4=ufc_fights['result']!='D'
method_mask=method_mask1&method_mask2&method_mask3&method_mask4

#forming total mask
mask=mask&maska&maskb&stance_mask_a&stance_mask_b&method_mask

#applies all masks resulting in ufc_fights_cleaned (annoying notation... clean the whole thing up)
ufc_fights_cleaned=ufc_fights[mask]

#Now lets get rid of the different types of decisions (all become DEC)
ufc_fights_cleaned['method'] = ufc_fights_cleaned['method'].replace(['U-DEC'],'DEC')
ufc_fights_cleaned['method'] = ufc_fights_cleaned['method'].replace(['S-DEC'],'DEC')
ufc_fights_cleaned['method'] = ufc_fights_cleaned['method'].replace(['M-DEC'],'DEC')

#change all columns to have float values
for col in ufc_fights_cleaned.columns:
    try:
        ufc_fights_cleaned[col]=ufc_fights_cleaned[col].astype(float)
    except:
        None

/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

In [108]:
record_statistics=[u'fighter_wins', 
                   u'fighter_losses',
                   u'fighter_L5Y_wins',
                   u'fighter_L5Y_losses', 
                   u'fighter_L2Y_wins', 
                   u'fighter_L2Y_losses',
                    u'fighter_ko_wins',
                   u'fighter_ko_losses',
                   u'fighter_L5Y_ko_wins',
                   u'fighter_L5Y_ko_losses',
                   u'fighter_L2Y_ko_wins',
                    u'fighter_L2Y_ko_losses',
                   u'fighter_sub_wins',
                   u'fighter_sub_losses',
                   u'fighter_L5Y_sub_wins',
                    u'fighter_L5Y_sub_losses', 
                   u'fighter_L2Y_sub_wins', 
                   u'fighter_L2Y_sub_losses',
                   u'opponent_wins', 
                   u'opponent_losses',
                   u'opponent_L5Y_wins', 
                   u'opponent_L5Y_losses', 
                   u'opponent_L2Y_wins', 
                   u'opponent_L2Y_losses', 
                    u'opponent_ko_wins', 
                   u'opponent_ko_losses', 
                   u'opponent_L5Y_ko_wins', 
                   u'opponent_L5Y_ko_losses', 
                   u'opponent_L2Y_ko_wins',
                    u'opponent_L2Y_ko_losses', 
                   u'opponent_sub_wins', 
                   u'opponent_sub_losses',
                   u'opponent_L5Y_sub_wins', 
                    u'opponent_L5Y_sub_losses', 
                   u'opponent_L2Y_sub_wins', 
                   u'opponent_L2Y_sub_losses']

physical_stats=[ u'fighter_age',
                u'fighter_height',
                    u'fighter_reach',  
                u'opponent_age',  
                u'opponent_height',
                    u'opponent_reach']

#here is the list of all stats available (besides stance), does not include names or result
punch_statistics=[    u'fighter_inf_knockdowns_avg',
                    u'fighter_inf_pass_avg',
                    u'fighter_inf_reversals_avg',
                    u'fighter_inf_sub_attempts_avg',
                    u'fighter_inf_takedowns_landed_avg',
                    u'fighter_inf_takedowns_attempts_avg',
                    u'fighter_inf_sig_strikes_landed_avg',
                    u'fighter_inf_sig_strikes_attempts_avg',
                    u'fighter_inf_total_strikes_landed_avg',
                    u'fighter_inf_total_strikes_attempts_avg',
                    u'fighter_inf_head_strikes_landed_avg',
                    u'fighter_inf_head_strikes_attempts_avg',
                    u'fighter_inf_body_strikes_landed_avg',
                    u'fighter_inf_body_strikes_attempts_avg',
                    u'fighter_inf_leg_strikes_landed_avg',
                    u'fighter_inf_leg_strikes_attempts_avg',
                    u'fighter_inf_distance_strikes_landed_avg',
                    u'fighter_inf_distance_strikes_attempts_avg',
                    u'fighter_inf_clinch_strikes_landed_avg',
                    u'fighter_inf_clinch_strikes_attempts_avg',
                    u'fighter_inf_ground_strikes_landed_avg',
                    u'fighter_inf_ground_strikes_attempts_avg',
                
                    u'fighter_abs_knockdowns_avg',
                    u'fighter_abs_pass_avg',
                    u'fighter_abs_reversals_avg',
                    u'fighter_abs_sub_attempts_avg',
                    u'fighter_abs_takedowns_landed_avg',
                    u'fighter_abs_takedowns_attempts_avg',
                    u'fighter_abs_sig_strikes_landed_avg',
                    u'fighter_abs_sig_strikes_attempts_avg',
                    u'fighter_abs_total_strikes_landed_avg',
                    u'fighter_abs_total_strikes_attempts_avg',
                    u'fighter_abs_head_strikes_landed_avg',
                    u'fighter_abs_head_strikes_attempts_avg',
                    u'fighter_abs_body_strikes_landed_avg',
                    u'fighter_abs_body_strikes_attempts_avg',
                    u'fighter_abs_leg_strikes_landed_avg',
                    u'fighter_abs_leg_strikes_attempts_avg',
                    u'fighter_abs_distance_strikes_landed_avg',
                    u'fighter_abs_distance_strikes_attempts_avg',
                    u'fighter_abs_clinch_strikes_landed_avg',
                    u'fighter_abs_clinch_strikes_attempts_avg',
                    u'fighter_abs_ground_strikes_landed_avg',
                    u'fighter_abs_ground_strikes_attempts_avg',
                    
                    u'opponent_inf_knockdowns_avg',
                    u'opponent_inf_pass_avg',
                    u'opponent_inf_reversals_avg',
                    u'opponent_inf_sub_attempts_avg',
                    u'opponent_inf_takedowns_landed_avg',
                    u'opponent_inf_takedowns_attempts_avg',
                    u'opponent_inf_sig_strikes_landed_avg',
                    u'opponent_inf_sig_strikes_attempts_avg',
                    u'opponent_inf_total_strikes_landed_avg',
                    u'opponent_inf_total_strikes_attempts_avg',
                    u'opponent_inf_head_strikes_landed_avg',
                    u'opponent_inf_head_strikes_attempts_avg',
                    u'opponent_inf_body_strikes_landed_avg',
                    u'opponent_inf_body_strikes_attempts_avg',
                    u'opponent_inf_leg_strikes_landed_avg',
                    u'opponent_inf_leg_strikes_attempts_avg',
                    u'opponent_inf_distance_strikes_landed_avg',
                    u'opponent_inf_distance_strikes_attempts_avg',
                    u'opponent_inf_clinch_strikes_landed_avg',
                    u'opponent_inf_clinch_strikes_attempts_avg',
                    u'opponent_inf_ground_strikes_landed_avg',
                    u'opponent_inf_ground_strikes_attempts_avg',
                
                    u'opponent_abs_knockdowns_avg',
                    u'opponent_abs_pass_avg',
                    u'opponent_abs_reversals_avg',
                    u'opponent_abs_sub_attempts_avg',
                    u'opponent_abs_takedowns_landed_avg',
                    u'opponent_abs_takedowns_attempts_avg',
                    u'opponent_abs_sig_strikes_landed_avg',
                    u'opponent_abs_sig_strikes_attempts_avg',
                    u'opponent_abs_total_strikes_landed_avg',
                    u'opponent_abs_total_strikes_attempts_avg',
                    u'opponent_abs_head_strikes_landed_avg',
                    u'opponent_abs_head_strikes_attempts_avg',
                    u'opponent_abs_body_strikes_landed_avg',
                    u'opponent_abs_body_strikes_attempts_avg',
                    u'opponent_abs_leg_strikes_landed_avg',
                    u'opponent_abs_leg_strikes_attempts_avg',
                    u'opponent_abs_distance_strikes_landed_avg',
                    u'opponent_abs_distance_strikes_attempts_avg',
                    u'opponent_abs_clinch_strikes_landed_avg',
                    u'opponent_abs_clinch_strikes_attempts_avg',
                    u'opponent_abs_ground_strikes_landed_avg',
                    u'opponent_abs_ground_strikes_attempts_avg']

#forming a dataframe where columns are differences between fighter and opponent fight statistics
mrs=len(record_statistics)/2
record_diff_2_df=pd.DataFrame()
for i in range(mrs):
    record_diff_2_df[record_statistics[i]+'_diff_2']=ufc_fights_cleaned[record_statistics[i]]-ufc_fights_cleaned[record_statistics[i+mrs]]
    
#Lets start building a dataframe of all the stats which we know perform best
ufc_best_df=pd.DataFrame()
ufc_best_df['fighter']=ufc_fights_cleaned['fighter']
ufc_best_df['opponent']=ufc_fights_cleaned['opponent']
ufc_best_df['result']=ufc_fights_cleaned['result']
ufc_best_df['method']=ufc_fights_cleaned['method']
ufc_best_df['date']=ufc_fights_cleaned['date']
for col in record_diff_2_df.columns:
    ufc_best_df[col]=record_diff_2_df[col]

#moving on to physical_stats:
physical_stats_diff_df=pd.DataFrame()
physical=[ [u'fighter_age',u'opponent_age'],[u'fighter_height',u'opponent_height'],[u'fighter_reach',u'opponent_reach']]
physical_stats_diff_df['age_diff']=ufc_fights_cleaned['fighter_age'].apply(float)-ufc_fights_cleaned['opponent_age'].apply(float)
physical_stats_diff_df['height_diff']=ufc_fights_cleaned['fighter_height'].apply(float)-ufc_fights_cleaned['opponent_height'].apply(float)
physical_stats_diff_df['reach_diff']=ufc_fights_cleaned['fighter_reach'].apply(float)-ufc_fights_cleaned['opponent_reach'].apply(float)
#ok so we keep ages absolute, but make differences for height and reach
ufc_best_df['fighter_age']=ufc_fights_cleaned['fighter_age']
ufc_best_df['opponent_age']=ufc_fights_cleaned['opponent_age']
#for some reason ages are stored as strings. Better if they are integers
ufc_best_df['fighter_age'] = ufc_best_df['fighter_age'].astype(int)
ufc_best_df['opponent_age'] = ufc_best_df['opponent_age'].astype(int)
#height and reach are best as differences
ufc_best_df['height_diff']=physical_stats_diff_df['height_diff']
ufc_best_df['reach_diff']=physical_stats_diff_df['reach_diff']

#This includes all the best choices in terms of taking differences
for col in punch_statistics:
    ufc_best_df[col]=ufc_fights_cleaned[col]

#adding stance to ufc_best_df
ufc_best_df['fighter_stance']=ufc_fights_cleaned['fighter_stance']
ufc_best_df['opponent_stance']=ufc_fights_cleaned['opponent_stance']

maybies=[
['fighter_wins_diff_2',
'fighter_losses_diff_2'],
['fighter_L5Y_wins_diff_2',
'fighter_L5Y_losses_diff_2',
'fighter_L2Y_wins_diff_2',
'fighter_L2Y_losses_diff_2'],
['fighter_L5Y_ko_wins_diff_2',
'fighter_L5Y_ko_losses_diff_2',
'fighter_L2Y_ko_wins_diff_2',
'fighter_L2Y_ko_losses_diff_2'],
['fighter_sub_losses_diff_2',
'fighter_L5Y_sub_wins_diff_2',
'fighter_L5Y_sub_losses_diff_2',
'fighter_L2Y_sub_wins_diff_2',
'fighter_L2Y_sub_losses_diff_2'],
['fighter_inf_reversals_avg',
'fighter_abs_reversals_avg',
'opponent_inf_reversals_avg',
'opponent_abs_reversals_avg'],
['fighter_inf_sub_attempts_avg',
'fighter_abs_sub_attempts_avg',
'opponent_inf_sub_attempts_avg',
'opponent_abs_sub_attempts_avg'],
['fighter_inf_takedowns_attempts_avg',
'fighter_abs_takedowns_attempts_avg',
'opponent_inf_takedowns_attempts_avg',
'opponent_abs_takedowns_attempts_avg'],
['fighter_inf_sig_strikes_landed_avg',
'fighter_abs_sig_strikes_landed_avg',
'opponent_inf_sig_strikes_landed_avg',
'opponent_abs_sig_strikes_landed_avg'],
['fighter_inf_sig_strikes_attempts_avg',
'fighter_abs_sig_strikes_attempts_avg',
'opponent_inf_sig_strikes_attempts_avg',
'opponent_abs_sig_strikes_attempts_avg'],
['fighter_inf_total_strikes_attempts_avg',
'fighter_abs_total_strikes_attempts_avg',
'opponent_inf_total_strikes_attempts_avg',
'opponent_abs_total_strikes_attempts_avg'],
['fighter_inf_head_strikes_attempts_avg',
'fighter_abs_head_strikes_attempts_avg',
'opponent_inf_head_strikes_attempts_avg',
'opponent_abs_head_strikes_attempts_avg'],
['fighter_inf_distance_strikes_landed_avg',
'fighter_abs_distance_strikes_landed_avg',
'opponent_inf_distance_strikes_landed_avg',
'opponent_abs_distance_strikes_landed_avg'],
['fighter_inf_distance_strikes_attempts_avg',
'fighter_abs_distance_strikes_attempts_avg',
'opponent_inf_distance_strikes_attempts_avg',
'opponent_abs_distance_strikes_attempts_avg'],
['fighter_inf_clinch_strikes_landed_avg',
'fighter_abs_clinch_strikes_landed_avg',
'opponent_inf_clinch_strikes_landed_avg',
'opponent_abs_clinch_strikes_landed_avg'],
]
keeps=[['fighter_ko_wins_diff_2'],
        ['fighter_inf_total_strikes_landed_avg',
        'opponent_inf_total_strikes_landed_avg',
        'fighter_abs_total_strikes_landed_avg',
        'opponent_abs_total_strikes_landed_avg'],
        ['fighter_age',
        'opponent_age'],
        ['fighter_sub_wins_diff_2'],
        ['fighter_inf_head_strikes_landed_avg',
        'fighter_abs_head_strikes_landed_avg',
        'opponent_inf_head_strikes_landed_avg',
        'opponent_abs_head_strikes_landed_avg'],
        ['fighter_ko_losses_diff_2'],
        ['fighter_inf_body_strikes_landed_avg',
        'fighter_inf_body_strikes_attempts_avg',
        'fighter_abs_body_strikes_landed_avg',
        'fighter_abs_body_strikes_attempts_avg',
        'opponent_inf_body_strikes_landed_avg',
        'opponent_inf_body_strikes_attempts_avg',
        'opponent_abs_body_strikes_landed_avg',
        'opponent_abs_body_strikes_attempts_avg'],
        ['fighter_inf_takedowns_landed_avg',
        'fighter_abs_takedowns_landed_avg',
        'opponent_inf_takedowns_landed_avg',
        'opponent_abs_takedowns_landed_avg'],
        ['fighter_inf_knockdowns_avg',
        'fighter_abs_knockdowns_avg',
        'opponent_inf_knockdowns_avg',
        'opponent_abs_knockdowns_avg'],
        ['fighter_inf_ground_strikes_attempts_avg',
        'fighter_abs_ground_strikes_attempts_avg',
        'opponent_inf_ground_strikes_attempts_avg',
        'opponent_abs_ground_strikes_attempts_avg',
        'fighter_inf_ground_strikes_landed_avg',
        'fighter_abs_ground_strikes_landed_avg',
        'opponent_inf_ground_strikes_landed_avg',
        'opponent_abs_ground_strikes_landed_avg'],
        ['fighter_inf_leg_strikes_landed_avg',
        'fighter_inf_leg_strikes_attempts_avg',
        'fighter_abs_leg_strikes_landed_avg',
        'fighter_abs_leg_strikes_attempts_avg',
        'opponent_inf_leg_strikes_landed_avg',
        'opponent_inf_leg_strikes_attempts_avg',
        'opponent_abs_leg_strikes_landed_avg',
        'opponent_abs_leg_strikes_attempts_avg'],
        ['fighter_inf_clinch_strikes_attempts_avg',
        'fighter_abs_clinch_strikes_attempts_avg',
        'opponent_inf_clinch_strikes_attempts_avg',
        'opponent_abs_clinch_strikes_attempts_avg'],
        ['fighter_inf_pass_avg',
        'fighter_abs_pass_avg',
        'opponent_inf_pass_avg',
        'opponent_abs_pass_avg'],
        ['fighter_stance',
        'opponent_stance',
        'height_diff',
        'reach_diff']]

ultimate_feature_list=['fighter_wins_diff_2',
 'fighter_losses_diff_2',
 'fighter_L5Y_wins_diff_2',
 'fighter_L5Y_losses_diff_2',
 'fighter_L2Y_wins_diff_2',
 'fighter_L2Y_losses_diff_2',
 'fighter_L5Y_ko_wins_diff_2',
 'fighter_L5Y_ko_losses_diff_2',
 'fighter_L2Y_ko_wins_diff_2',
 'fighter_L2Y_ko_losses_diff_2',
 'fighter_sub_losses_diff_2',
 'fighter_L5Y_sub_wins_diff_2',
 'fighter_L5Y_sub_losses_diff_2',
 'fighter_L2Y_sub_wins_diff_2',
 'fighter_L2Y_sub_losses_diff_2',
 'fighter_inf_reversals_avg',
 'fighter_abs_reversals_avg',
 'opponent_inf_reversals_avg',
 'opponent_abs_reversals_avg',
 'fighter_inf_takedowns_attempts_avg',
 'fighter_abs_takedowns_attempts_avg',
 'opponent_inf_takedowns_attempts_avg',
 'opponent_abs_takedowns_attempts_avg',
 'fighter_ko_wins_diff_2',
 'fighter_age',
 'opponent_age',
 'fighter_sub_wins_diff_2',
 'fighter_ko_losses_diff_2',
 'fighter_inf_body_strikes_landed_avg',
 'fighter_inf_body_strikes_attempts_avg',
 'fighter_abs_body_strikes_landed_avg',
 'fighter_abs_body_strikes_attempts_avg',
 'opponent_inf_body_strikes_landed_avg',
 'opponent_inf_body_strikes_attempts_avg',
 'opponent_abs_body_strikes_landed_avg',
 'opponent_abs_body_strikes_attempts_avg',
 'fighter_inf_takedowns_landed_avg',
 'fighter_abs_takedowns_landed_avg',
 'opponent_inf_takedowns_landed_avg',
 'opponent_abs_takedowns_landed_avg']

#in the file ufc_finding_the_ultimate_model, we found the following to be (among the) optimal possibilities
modified_ultimate_feature_list=['fighter_wins_diff_2',
 'fighter_losses_diff_2',
 'fighter_L5Y_wins_diff_2',
 'fighter_L5Y_losses_diff_2',
 'fighter_L2Y_wins_diff_2',
 'fighter_L2Y_losses_diff_2',
 'fighter_L5Y_ko_wins_diff_2',
 'fighter_L5Y_ko_losses_diff_2',
 'fighter_L2Y_ko_wins_diff_2',
 'fighter_L2Y_ko_losses_diff_2',
 'fighter_sub_losses_diff_2',
 'fighter_L5Y_sub_wins_diff_2',
 'fighter_L5Y_sub_losses_diff_2',
 'fighter_L2Y_sub_wins_diff_2',
 'fighter_L2Y_sub_losses_diff_2',
 'fighter_inf_reversals_avg',
 'fighter_abs_reversals_avg',
 'opponent_inf_reversals_avg',
 'opponent_abs_reversals_avg',
 'fighter_inf_takedowns_attempts_avg',
 'fighter_abs_takedowns_attempts_avg',
 'opponent_inf_takedowns_attempts_avg',
 'opponent_abs_takedowns_attempts_avg',
 'fighter_ko_wins_diff_2',
 'fighter_age',
 'opponent_age',
 'fighter_sub_wins_diff_2',
 'fighter_ko_losses_diff_2',
 'fighter_inf_body_strikes_landed_avg',
 'fighter_inf_body_strikes_attempts_avg',
 'fighter_abs_body_strikes_landed_avg',
 'opponent_inf_body_strikes_landed_avg',
 'opponent_inf_body_strikes_attempts_avg',
 'opponent_abs_body_strikes_landed_avg',
 'fighter_inf_takedowns_landed_avg',
 'fighter_abs_takedowns_landed_avg',
 'opponent_inf_takedowns_landed_avg',
 'opponent_abs_takedowns_landed_avg']

maybe_comp=[]
for r in [i for i in range(14) if not i in (0, 1, 2, 3, 4, 6)]:
    maybe_comp.append(maybies[r])
    
keeps_comp=[]
for i in (1,4,8,9,10,11,12,13):
    keeps_comp.append(keeps[i])

add_back=maybe_comp+keeps_comp

In [109]:
model=LogisticRegression()

In [122]:
len(ufc_best_df['result'])

3620

In [121]:
Xo=ufc_best_df[ufc_best_df.columns[5:]]
yo=ufc_best_df['result']
model_score=cross_val_score(model,Xo,yo,cv=8).mean()
model_score

0.6132648116769994

In [111]:
#lets go ahead and fit the model so we can make some predictions (its over 75 and not overfit!)
model.fit(Xo,yo)
model.score(Xo,yo)

0.6406077348066298

In [58]:
model.predict([ufc_best_df[best_best].iloc[100]])

array(['L'], dtype=object)

In [13]:
test_best=['fighter','opponent','result','method','date']+best_best

In [14]:
ufc_best_df[test_best].iloc[100][:4]

fighter        Cory Sandhagen
opponent    Aljamain Sterling
result                      L
method                    SUB
Name: 142, dtype: object

In [15]:
model.predict([ufc_best_df[best_best].iloc[100]])

array(['L'], dtype=object)

In [ ]:
ufc_best_df[test_best].iloc[101][:4]

In [ ]:
model.predict([ufc_best_df[best_best].iloc[101]])

In [ ]:
ufc_predict('Anthony Rocco Martin','Neil Magny','June 06, 2020','June 06, 2020')

In [ ]:
ufc_best_df[test_best].iloc[102][:4]

In [ ]:
model.predict([ufc_best_df[best_best].iloc[102]])

In [ ]:
mlp.predict([ufc_fights_cleaned[candidate].iloc[102]])

In [ ]:
ufc_predict('Eddie Wineland',"Sean O'Malley",'June 06, 2020','June 06, 2020')

In [ ]:
ufc_best_df[test_best].iloc[0][0]

In [ ]:
ufc_best_df[test_best]['result'].iloc[100]

In [17]:
#testing accuracy by hand just to be sure... seems right
results=[]
for i in range(2000):
    try:
        prediction=model.predict([ufc_best_df[best_best].iloc[i]])[0]
        result=ufc_best_df[method_mask]['result'].iloc[i]
        results.append(prediction==result)
    except:
        None

/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [18]:
sum(results)/float(len(results))

0.75

In [ ]:
#again based on the building the ultimate mlp file, the following is optimal for mlp

In [19]:
#here we list all of the possible statistics. Next lets identify based on intuition which could help predict method
mlp_stats_grouped=[[u'fighter_L5Y_wins',
                   u'fighter_L5Y_losses', 
                   u'fighter_L2Y_wins', 
                   u'fighter_L2Y_losses',
                   u'opponent_L5Y_wins', 
                   u'opponent_L5Y_losses', 
                   u'opponent_L2Y_wins', 
                   u'opponent_L2Y_losses'], 
                   [u'fighter_ko_wins',
                   u'fighter_ko_losses',
                   u'opponent_ko_wins', 
                   u'opponent_ko_losses'],
                   [u'fighter_L5Y_ko_wins',
                   u'fighter_L5Y_ko_losses',
                   u'fighter_L2Y_ko_wins',
                    u'fighter_L2Y_ko_losses',
                   u'opponent_L5Y_ko_wins', 
                   u'opponent_L5Y_ko_losses', 
                   u'opponent_L2Y_ko_wins',
                    u'opponent_L2Y_ko_losses'],
                   [u'fighter_sub_wins',
                   u'fighter_sub_losses',
                   u'opponent_sub_wins', 
                   u'opponent_sub_losses'],
                   [u'fighter_L5Y_sub_wins',
                    u'fighter_L5Y_sub_losses', 
                   u'fighter_L2Y_sub_wins', 
                   u'fighter_L2Y_sub_losses',
                   u'opponent_L5Y_sub_wins', 
                    u'opponent_L5Y_sub_losses', 
                   u'opponent_L2Y_sub_wins', 
                   u'opponent_L2Y_sub_losses'],
                    [u'fighter_age', 
                     u'opponent_age'],
                    [u'fighter_height',
                     u'fighter_reach', 
                    u'opponent_height',
                     u'opponent_reach'],
                    [u'fighter_inf_knockdowns_avg',
                    u'fighter_abs_knockdowns_avg',
                    u'opponent_inf_knockdowns_avg',
                    u'opponent_abs_knockdowns_avg'],
                    [u'fighter_inf_pass_avg',
                    u'fighter_abs_pass_avg',
                    u'opponent_inf_pass_avg',
                    u'opponent_abs_pass_avg'],
                    [u'fighter_inf_reversals_avg',
                    u'fighter_abs_reversals_avg',
                    u'opponent_inf_reversals_avg',
                    u'opponent_abs_reversals_avg'],
                    [u'fighter_inf_sub_attempts_avg',
                    u'fighter_abs_sub_attempts_avg',
                    u'opponent_inf_sub_attempts_avg',
                    u'opponent_abs_sub_attempts_avg'],
                    [u'fighter_inf_takedowns_landed_avg',
                    u'fighter_inf_takedowns_attempts_avg',
                    u'fighter_abs_takedowns_landed_avg',
                    u'fighter_abs_takedowns_attempts_avg',
                    u'opponent_inf_takedowns_landed_avg',
                    u'opponent_inf_takedowns_attempts_avg',
                    u'opponent_abs_takedowns_landed_avg',
                    u'opponent_abs_takedowns_attempts_avg'],
                    [u'fighter_inf_sig_strikes_landed_avg',
                    u'fighter_inf_sig_strikes_attempts_avg',
                    u'fighter_abs_sig_strikes_landed_avg',
                    u'fighter_abs_sig_strikes_attempts_avg',
                    u'opponent_inf_sig_strikes_landed_avg',
                    u'opponent_inf_sig_strikes_attempts_avg',
                    u'opponent_abs_sig_strikes_landed_avg',
                    u'opponent_abs_sig_strikes_attempts_avg'],
                    [u'fighter_inf_total_strikes_landed_avg',
                    u'fighter_inf_total_strikes_attempts_avg',
                    u'fighter_abs_total_strikes_landed_avg',
                    u'fighter_abs_total_strikes_attempts_avg',
                    u'opponent_inf_total_strikes_landed_avg',
                    u'opponent_inf_total_strikes_attempts_avg',
                    u'opponent_abs_total_strikes_landed_avg',
                    u'opponent_abs_total_strikes_attempts_avg'],
                    [u'fighter_inf_head_strikes_landed_avg',
                    u'fighter_inf_head_strikes_attempts_avg',
                    u'fighter_abs_head_strikes_landed_avg',
                    u'fighter_abs_head_strikes_attempts_avg',
                    u'opponent_inf_head_strikes_landed_avg',
                    u'opponent_inf_head_strikes_attempts_avg',
                    u'opponent_abs_head_strikes_landed_avg',
                    u'opponent_abs_head_strikes_attempts_avg'],
                    [u'fighter_inf_body_strikes_landed_avg',
                    u'fighter_inf_body_strikes_attempts_avg',
                    u'fighter_abs_body_strikes_landed_avg',
                    u'fighter_abs_body_strikes_attempts_avg',
                    u'opponent_inf_body_strikes_landed_avg',
                    u'opponent_inf_body_strikes_attempts_avg',
                    u'opponent_abs_body_strikes_landed_avg',
                    u'opponent_abs_body_strikes_attempts_avg'],
                    [u'fighter_inf_leg_strikes_landed_avg',
                    u'fighter_inf_leg_strikes_attempts_avg',
                    u'fighter_abs_leg_strikes_landed_avg',
                    u'fighter_abs_leg_strikes_attempts_avg',
                    u'opponent_inf_leg_strikes_landed_avg',
                    u'opponent_inf_leg_strikes_attempts_avg',
                    u'opponent_abs_leg_strikes_landed_avg',
                    u'opponent_abs_leg_strikes_attempts_avg'],
                    [u'fighter_inf_distance_strikes_landed_avg',
                    u'fighter_inf_distance_strikes_attempts_avg',
                    u'fighter_abs_distance_strikes_landed_avg',
                    u'fighter_abs_distance_strikes_attempts_avg',
                    u'opponent_inf_distance_strikes_landed_avg',
                    u'opponent_inf_distance_strikes_attempts_avg',
                    u'opponent_abs_distance_strikes_landed_avg',
                    u'opponent_abs_distance_strikes_attempts_avg'],
                    [u'fighter_inf_clinch_strikes_landed_avg',
                    u'fighter_inf_clinch_strikes_attempts_avg',
                    u'fighter_abs_clinch_strikes_landed_avg',
                    u'fighter_abs_clinch_strikes_attempts_avg',
                    u'opponent_inf_clinch_strikes_landed_avg',
                    u'opponent_inf_clinch_strikes_attempts_avg',
                    u'opponent_abs_clinch_strikes_landed_avg',
                    u'opponent_abs_clinch_strikes_attempts_avg'],
                    [u'fighter_inf_ground_strikes_landed_avg',
                    u'fighter_inf_ground_strikes_attempts_avg',
                    u'fighter_abs_ground_strikes_landed_avg',
                    u'fighter_abs_ground_strikes_attempts_avg',
                    u'opponent_inf_ground_strikes_landed_avg',
                    u'opponent_inf_ground_strikes_attempts_avg',
                    u'opponent_abs_ground_strikes_landed_avg',
                    u'opponent_abs_ground_strikes_attempts_avg']
                    ]
len(mlp_stats_grouped)

mlp_add_backs_refined=[
 [u'fighter_age', u'opponent_age'],
 [u'fighter_inf_knockdowns_avg',
  u'opponent_inf_knockdowns_avg'],
  [u'fighter_abs_knockdowns_avg',
  u'opponent_abs_knockdowns_avg'],
 [u'fighter_inf_pass_avg',
  u'fighter_abs_pass_avg'],
  [u'opponent_inf_pass_avg',
  u'opponent_abs_pass_avg'],
 [u'fighter_inf_reversals_avg',
  u'opponent_inf_reversals_avg'],
  [u'fighter_abs_reversals_avg',
  u'opponent_abs_reversals_avg'],
 [u'fighter_inf_sub_attempts_avg',
  u'opponent_inf_sub_attempts_avg'],
  [u'fighter_abs_sub_attempts_avg',
  u'opponent_abs_sub_attempts_avg'],
 [u'fighter_inf_takedowns_landed_avg',
  u'opponent_inf_takedowns_landed_avg'],
  [u'fighter_abs_takedowns_landed_avg',
  u'opponent_abs_takedowns_landed_avg'],
  [u'fighter_inf_takedowns_attempts_avg',
   u'opponent_inf_takedowns_attempts_avg'],
  [u'fighter_abs_takedowns_attempts_avg',
  u'opponent_abs_takedowns_attempts_avg'],
 [u'fighter_inf_sig_strikes_landed_avg',
  u'opponent_inf_sig_strikes_landed_avg'],
  [u'fighter_abs_sig_strikes_landed_avg',
  u'opponent_abs_sig_strikes_landed_avg'],
  [u'fighter_inf_sig_strikes_attempts_avg',
   u'opponent_inf_sig_strikes_attempts_avg'],
  [u'fighter_abs_sig_strikes_attempts_avg',
  u'opponent_abs_sig_strikes_attempts_avg'],
 [u'fighter_inf_total_strikes_landed_avg',
  u'opponent_inf_total_strikes_landed_avg'],
  [u'fighter_abs_total_strikes_landed_avg',
  u'opponent_abs_total_strikes_landed_avg'],
  [u'fighter_inf_total_strikes_attempts_avg',
   u'opponent_inf_total_strikes_attempts_avg'],
  [u'fighter_abs_total_strikes_attempts_avg',
  u'opponent_abs_total_strikes_attempts_avg'],
 [u'fighter_inf_body_strikes_landed_avg',
  u'opponent_inf_body_strikes_landed_avg'],
  [u'fighter_abs_body_strikes_landed_avg',
  u'opponent_abs_body_strikes_landed_avg'],
  [u'fighter_inf_body_strikes_attempts_avg',
   u'opponent_inf_body_strikes_attempts_avg'],
  [u'fighter_abs_body_strikes_attempts_avg',
  u'opponent_abs_body_strikes_attempts_avg'],
 [u'fighter_inf_leg_strikes_landed_avg',
  u'opponent_inf_leg_strikes_landed_avg'],
  [u'fighter_abs_leg_strikes_landed_avg',
  u'opponent_abs_leg_strikes_landed_avg'],
  [u'fighter_inf_leg_strikes_attempts_avg',
   u'opponent_inf_leg_strikes_attempts_avg'],
  [u'fighter_abs_leg_strikes_attempts_avg',
  u'opponent_abs_leg_strikes_attempts_avg'],
 [u'fighter_inf_distance_strikes_landed_avg',
  u'opponent_inf_distance_strikes_landed_avg'],
  [u'fighter_abs_distance_strikes_landed_avg',
  u'opponent_abs_distance_strikes_landed_avg'],
  [u'fighter_inf_distance_strikes_attempts_avg',
   u'opponent_inf_distance_strikes_attempts_avg'],
  [u'fighter_abs_distance_strikes_attempts_avg',
  u'opponent_abs_distance_strikes_attempts_avg'],
 [u'fighter_inf_ground_strikes_landed_avg',
  u'opponent_inf_ground_strikes_landed_avg'],
  [u'fighter_abs_ground_strikes_landed_avg',
  u'opponent_abs_ground_strikes_landed_avg'],
  [u'fighter_inf_ground_strikes_attempts_avg',
   u'opponent_inf_ground_strikes_attempts_avg'],
  [u'fighter_abs_ground_strikes_attempts_avg',
  u'opponent_abs_ground_strikes_attempts_avg']]

mlp_stats=[]
for stat in mlp_stats_grouped:
    mlp_stats.extend(stat)

mlp_base_set=[]
for i in (1,2,3,14,18):
    mlp_base_set.extend(mlp_stats_grouped[i])
    
mlp_base_set_grouped=[]
for i in (1,2,3,14,18):
    mlp_base_set_grouped.append(mlp_stats_grouped[i])
    
mlp_add_backs=[]
for i in (0,4,5,6,7,8,9,10,11,12,13,15,16,17,19):
    mlp_add_backs.append(mlp_stats_grouped[i]) 

mlp=MLPClassifier(max_iter=2500,alpha=.001,solver='sgd',random_state=3)

#removing from mlp_add_backs and adding to mlp_base_set
mlp_base_set.extend(mlp_add_backs[1])
mlp_add_backs.remove(mlp_add_backs[1])

In [20]:
candidate=[]
candidate.extend(mlp_base_set)
candidate.extend(mlp_add_backs_refined[21])

In [21]:
#change all columns to have float values
for col in ufc_fights_cleaned[candidate].columns:
    ufc_fights_cleaned[col]=ufc_fights_cleaned[col].astype(float)

/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [22]:
#DEE_EE_EECENT
Xp=ufc_fights_cleaned[candidate].iloc[0:2091]
yp=ufc_fights_cleaned.iloc[0:2091]['method']
mlp_score=cross_val_score(mlp,Xp,yp,cv=10).mean()
mlp_score

0.6732274031587533

In [23]:
#fitting mlp (obviously mlp is always overfit, the above score is the accurate one)
#BIG QUESTION. WHY ISN'T THE MODEL PERFORMING THIS WAY WHEN I CHECK IT ON THE TRAINING DATA
#It is performing much closer to the cross_val_score...
mlp.fit(Xp,yp)
mlp.score(Xp,yp)

0.896700143472023

In [24]:
#unfortunately some mistakes makes this no longer true
#HERE IS THE OVERALL PROBABILITY OF PICKING BOTH THE WINNER AND THE METHOD CORRECTLY (this was exactly my goal, .5)

In [25]:
overall_score=model_score*mlp_score
overall_score

0.500012714860965

In [ ]:
fighter=ufc_best_df[test_best].iloc[200][0]

In [ ]:
opponent=ufc_best_df[test_best].iloc[200][1]

In [ ]:
ufc_best_df[test_best].iloc[200][2]

In [ ]:
ufc_best_df[test_best].iloc[200][3]

In [ ]:
datte=ufc_best_df[test_best].iloc[200][4]

In [ ]:
datte

In [ ]:
opponent

In [91]:
accuracy_test={}
model_correct=0
mlp_correct=0
overfit_model_correct=0
overfit_mlp_correct=0
for i in range(100):
    fighter=ufc_best_df[test_best].iloc[i][0]
    opponent=ufc_best_df[test_best].iloc[i][1]
    result=ufc_best_df[test_best].iloc[i][2]
    method=ufc_best_df[test_best].iloc[i][3]
    datte=ufc_best_df[test_best].iloc[i][4]
    overfit_model_prediction=model.predict([ufc_best_df[best_best].iloc[i]])[0]
    overfit_mlp_prediction=mlp.predict([ufc_fights_cleaned[candidate].iloc[i]])[0]
    model_prediction=model.predict([model_prediction_tuple(fighter,opponent,datte,datte)])[0]
    mlp_prediction=mlp.predict([mlp_prediction_tuple(fighter,opponent,datte,datte)])[0]
    accuracy_test[i]=[result,method,overfit_model_prediction,overfit_mlp_prediction,model_prediction,mlp_prediction]
    overfit_model_correct+=(result==overfit_model_prediction)
    overfit_mlp_correct+=(method==overfit_mlp_prediction)
    model_correct+=(result==model_prediction)
    mlp_correct+=(method==mlp_prediction)
    overfit_model_acc=overfit_model_correct/float(i+1)
    overfit_mlp_acc=overfit_mlp_correct/float(i+1)
    model_acc=model_correct/float(i+1)
    mlp_acc=mlp_correct/float(i+1)
    print('fight: '+fighter+', '+opponent+'. '
        +' res/meth: '+str([result,method])
          + ' overfit: '+str([overfit_model_prediction,overfit_mlp_prediction])
    + ' pred: '+str([model_prediction,mlp_prediction])
          + ' overfit_acc: '+str([overfit_model_acc,overfit_mlp_acc])
         + ' acc: '+str([model_acc,mlp_acc]))

fight: Alistair Overeem, Augusto Sakai.  res/meth: ['W', 'KO/TKO'] overfit: ['L', 'KO/TKO'] pred: ['L', 'KO/TKO'] overfit_acc: [0.0, 1.0] acc: [0.0, 1.0]
fight: Ovince Saint Preux, Alonzo Menifield.  res/meth: ['W', 'KO/TKO'] overfit: ['W', 'KO/TKO'] pred: ['W', 'KO/TKO'] overfit_acc: [0.5, 1.0] acc: [0.5, 1.0]
fight: Michel Pereira, Zelim Imadaev.  res/meth: ['W', 'SUB'] overfit: ['W', 'DEC'] pred: ['W', 'DEC'] overfit_acc: [0.6666666666666666, 0.6666666666666666] acc: [0.6666666666666666, 0.6666666666666666]
fight: Andre Muniz, Bartosz Fabinski.  res/meth: ['W', 'SUB'] overfit: ['L', 'DEC'] pred: ['L', 'DEC'] overfit_acc: [0.5, 0.5] acc: [0.5, 0.5]
fight: Montana De La Rosa, Viviane Araujo.  res/meth: ['L', 'DEC'] overfit: ['W', 'DEC'] pred: ['W', 'DEC'] overfit_acc: [0.4, 0.6] acc: [0.4, 0.6]
fight: Cole Smith, Hunter Azure.  res/meth: ['L', 'DEC'] overfit: ['L', 'DEC'] pred: ['L', 'DEC'] overfit_acc: [0.5, 0.6666666666666666] acc: [0.5, 0.6666666666666666]
fight: Anthony Smith, Ale

KeyboardInterrupt: 

In [ ]:
#THIS IS FUCKING TRASH, I've seen enough (i'm very confused why overfit isn't at [.75,.9])
#maybe i need to reanalyze how i am training after fixing the data frame.
#perhaps the model gives too much credit to the (cheating stats)

In [ ]:
#Explanation for under performance. I realized that sub_wins/losses and ko_wins/losses were calculated
#in ufc_fights_crap and ufc_fights as total number of wins/losses, not wins/losses before date (by a big mistake)
#this means that the models were trained with 'future knowing' data. But now we are plugging in data that does
#not see the future since I fixed the mistake. We'll see what happens once I fix the data set and retrain.
#I should now go through the data sets and make sure there are no other big mistakes... or future seeing (cheating)
#note the overfitness of the method (its a neural network)... on actual fight predictions, this accuracy will be
#a bit less accurate than the winner/loser prediction
#I COULD TRY retraining now and dropping the cheating stats, and see what we get

In [ ]:
#idea: lets add L5Y average weight

In [35]:
fighter=ufc_best_df[test_best].iloc[3][0]
opponent=ufc_best_df[test_best].iloc[3][1]
result=ufc_best_df[test_best].iloc[3][2]
method=ufc_best_df[test_best].iloc[3][3]
datte=ufc_best_df[test_best].iloc[3][4]

In [47]:
datte

'September 05, 2020'

In [36]:
print (fighter,opponent,result,method,datte)

('Andre Muniz', 'Bartosz Fabinski', 'W', 'SUB', 'September 05, 2020')


In [ ]:
overfit_model_prediction=model.predict([ufc_best_df[best_best].iloc[3]])[0]
overfit_mlp_prediction=mlp.predict([ufc_fights_cleaned[candidate].iloc[3]])[0]

In [43]:
frame_tuple=ufc_best_df[best_best].iloc[3]

In [46]:
frame_tuple

fighter_wins_diff_2                           -2.000000
fighter_losses_diff_2                         -1.000000
fighter_L5Y_wins_diff_2                       -1.000000
fighter_L5Y_losses_diff_2                     -1.000000
fighter_L2Y_wins_diff_2                        1.000000
fighter_L2Y_losses_diff_2                     -1.000000
fighter_L5Y_ko_wins_diff_2                     0.000000
fighter_L5Y_ko_losses_diff_2                   0.000000
fighter_L2Y_ko_wins_diff_2                     0.000000
fighter_L2Y_ko_losses_diff_2                   0.000000
fighter_sub_losses_diff_2                     -1.000000
fighter_L5Y_sub_wins_diff_2                    0.000000
fighter_L5Y_sub_losses_diff_2                 -1.000000
fighter_L2Y_sub_wins_diff_2                    0.000000
fighter_L2Y_sub_losses_diff_2                 -1.000000
fighter_inf_reversals_avg                      0.000000
fighter_abs_reversals_avg                      0.066667
opponent_inf_reversals_avg                     0

In [87]:
generated_tuple=model_prediction_tuple(fighter,opponent,'September 04, 2020','September 04, 2020')

In [ ]:
#it seems the generated tuple is counting the sub loss for the fight...
#Figured out the issue. The sub_losses function was stupidly defined to count ALL sub losses
#still not sure why it was counted differently in the data frame... actually must be because we generated 
#the data frame actually
#before the fight happened!

In [88]:
for i in range(len(generated_tuple)):
    print(frame_tuple[i],generated_tuple[i])

(-2.0, -2.0)
(-1.0, -1.0)
(-1.0, -1.0)
(-1.0, -1.0)
(1.0, 1.0)
(-1.0, -1.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(-1.0, -1.0)
(0.0, 0.0)
(-1.0, -1.0)
(0.0, 0.0)
(-1.0, -1.0)
(0.0, 0.0)
(0.06666666666666668, 0.06666666666666667)
(0.0, 0.0)
(0.0, 0.0)
(0.4666666666666667, 0.4666666666666667)
(0.06666666666666668, 0.06666666666666667)
(0.6734250543084721, 0.6734250543084721)
(0.10861694424330197, 0.10861694424330195)
(0.0, 0.0)
(30.0, 30.0)
(34.0, 34.0)
(0.0, 0.0)
(0.0, 0.0)
(0.5333333333333333, 0.5333333333333333)
(0.6666666666666666, 0.6666666666666666)
(0.3333333333333333, 0.3333333333333333)
(0.4666666666666667, 0.4666666666666667)
(0.6082548877624909, 0.6082548877624909)
(0.6517016654598118, 0.6517016654598118)
(0.10861694424330197, 0.10861694424330195)
(0.19551049963794356, 0.19551049963794354)
(0.2, 0.2)
(0.06666666666666668, 0.06666666666666667)
(0.4779145546705286, 0.4779145546705286)
(0.02172338884866039, 0.02172338884866039)
(2.0, 2.0)
(2.333333333333333, 2.3333333333333

In [89]:
model.predict([frame_tuple])

array(['L'], dtype=object)

In [90]:
model.predict([generated_tuple])

array(['L'], dtype=object)

In [67]:
mlp_df_tuple=ufc_fights_cleaned.iloc[3]

In [68]:
mlp_df_tuple

date                                          September 05, 2020
division                                            Middleweight
fighter                                              Andre Muniz
opponent                                        Bartosz Fabinski
result                                                         W
method                                                       SUB
fighter_wins                                                   1
fighter_losses                                                 0
fighter_age                                                   30
fighter_height                                            185.42
fighter_reach                                             198.12
fighter_L5Y_wins                                               1
fighter_L5Y_losses                                             0
fighter_L2Y_wins                                               1
fighter_L2Y_losses                                             0
fighter_ko_wins          

In [80]:
mlp_generated_tuple=mlp_prediction_tuple(fighter,opponent,datte,datte)

In [81]:
sub_wins('Andre Muniz','September 04, 2020')

0.0

In [83]:
for i in range(len(mlp_generated_tuple)):
    print(ufc_fights_cleaned[candidate].iloc[3][i],mlp_generated_tuple[i])

(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(1.0, 1.0)
(0.8, 0.8)
(2.0, 2.0)
(1.3333333333333333, 1.3333333333333333)
(2.333333333333333, 2.3333333333333335)
(1.3685734974656047, 1.3685734974656047)
(2.2592324402606807, 2.2592324402606807)
(1.0644460535843592, 1.0644460535843592)
(2.541636495293266, 2.541636495293266)
(0.06666666666666668, 0.06666666666666667)
(0.06666666666666668, 0.06666666666666667)
(0.0, 0.0)
(0.0, 0.0)
(0.9123823316437364, 0.9123823316437365)
(1.0209992758870383, 1.0209992758870383)
(0.4779145546705286, 0.4779145546705286)
(0.5865314989138306, 0.5865314989138306)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(1.0, 1.0)
(0.0, 0.0)
(1.0, 1.0)
(0.5333333333333333, 0.5333333333333333)
(0.6082548877624909, 0.6082548877624909)


In [40]:
mlp_prediction_tuple(fighter,opponent,datte,datte)

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 2.0,
 0.8,
 2.0,
 1.3333333333333333,
 2.3333333333333335,
 1.3685734974656047,
 2.2592324402606807,
 1.0644460535843592,
 2.541636495293266,
 0.06666666666666667,
 0.06666666666666667,
 0.0,
 0.0,
 0.9123823316437365,
 1.0209992758870383,
 0.4779145546705286,
 0.5865314989138306,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.5333333333333333,
 0.6082548877624909]

In [ ]:
model_prediction=model.predict([model_prediction_tuple(fighter,opponent,datte,datte)])[0]
mlp_prediction=mlp.predict([mlp_prediction_tuple(fighter,opponent,datte,datte)])[0]
accuracy_test[i]=[result,method,overfit_model_prediction,overfit_mlp_prediction,model_prediction,mlp_prediction]
overfit_model_correct+=(result==overfit_model_prediction)
overfit_mlp_correct+=(method==overfit_mlp_prediction)
model_correct+=(result==model_prediction)
mlp_correct+=(method==mlp_prediction)
overfit_model_acc=overfit_model_correct/float(i+1)
overfit_mlp_acc=overfit_mlp_correct/float(i+1)
model_acc=model_correct/float(i+1)
mlp_acc=mlp_correct/float(i+1)

In [ ]:
#after running for quite awhile this should end up at [.75,.9] for both acc and overfit_acc 
#figure out what is going wrong with the ufc_predict function
#after this runs for a bit lets check back on the ufc_predict values and see if they agree with the predictions here
#I mean, it would be stupid if they didn't, they are literally the same thing...
#There is clearly a problem with the overfit accuracy...

In [ ]:
#THE OVERALL SCORE IS .5000... THIS IS INSANE HONESLTLY. THIS MEANS HALF THE TIME I WILL GET BOTH THE 
#WINNER AND METHOD OF VICTORY CORRECTLY... WITH +200 AVERAGE ODDS THIS MEANS OUT OF EVERY TWO FIGHTS
#I SPEND 100 AND GET 200 BACK. So basically if I bet x dollars per fight, I will make x/2 dollars per fight on average.
#Ok lets get to betting (if the average is really +200). Lets check

In [ ]:
#THE PROBLEM IS, THIS IS NOT THE SCORE I GET WHEN I EVEN TEST ON (NEARLY) THE TRAINING DATA ITSELF
#SOMETHING IS GOING HORRIBLY WRONG...
#WHEN I TEST ON EXACT TRAINING DATA, I GET THE RIGHT THING, BUT IT SEEMS FOR A VERY SLIGHT PERTURBATION,
#IT THROWS EVERYTHING OFF, AND I AM BACK DOWN TO 50% ACCURACY NEARLY

In [ ]:
time ufc_predict('Cole Smith','Hunter Azure', 'September 6, 2020', 'September 6, 2020')
#completely correct

In [ ]:
#making a dictionary to track accuracy
acc_dic={}
acc_dic[0]=[1,1]

In [ ]:
time ufc_predict('Viviane Araujo','Montana De La Rosa', 'September 6, 2020', 'September 6, 2020')
#wrong winner right method

In [ ]:
acc_dic[1]=[0,1]

In [ ]:
time ufc_predict('Brian Kelleher','Ray Rodriguez', 'September 6, 2020', 'September 6, 2020')
#wrong winner wrong method

In [ ]:
acc_dic[2]=[0,0]

In [ ]:
time ufc_predict('Andre Muniz','Bartosz Fabinski', 'September 6, 2020', 'September 6, 2020')
#wrong winner wrong method

In [ ]:
acc_dic[3]=[0,0]

In [ ]:
time ufc_predict('Michel Pereira','Zelim Imadaev', 'September 6, 2020', 'September 6, 2020')
#right winner wrong method

In [ ]:
acc_dic[4]=[1,0]

In [ ]:
time ufc_predict('Ovince Saint Preux','Alonzo Menifield', 'September 6, 2020', 'September 6, 2020')
#right in both

In [ ]:
acc_dic[5]=[1,1]

In [ ]:
time ufc_predict('Alistair Overeem','Augusto Sakai', 'September 6, 2020', 'September 6, 2020')
#wrong winner right method
#WHATS GOING ON. I DIDN'T CHANGE THE MODEL BUT NOW ITS SAYING ALISTAIR WINS

In [ ]:
acc_dic[6]=[0,1]

In [ ]:
time ufc_predict('Ricardo Lamas','Bill Algeo', 'August 30, 2020', 'August 30, 2020')
#right on both

In [ ]:
acc_dic[7]=[1,1]

In [ ]:
time ufc_predict('Ji Yeon Kim','Alexa Grasso', 'August 30, 2020', 'August 30, 2020')
#right on both

In [ ]:
acc_dic[8]=[1,1]

In [ ]:
time ufc_predict('Robbie Lawler','Neil Magny', 'August 30, 2020', 'August 30, 2020')
#right on both

In [ ]:
acc_dic[9]=[1,1]

In [ ]:
time ufc_predict('Anthony Smith','Aleksandar Rakic', 'August 30, 2020', 'August 30, 2020')
#right on both

In [ ]:
acc_dic[10]=[1,1]

In [ ]:
time ufc_predict('Daniel Rodriguez','Dwight Grant', 'August 23, 2020', 'August 23, 2020')
#wrong on both

In [ ]:
acc_dic[11]=[0,0]

In [ ]:
time ufc_predict('Mariya Agapova','Shana Dobson', 'August 23, 2020', 'August 23, 2020')
#wrong on both

In [ ]:
acc_dic[12]=[0,0]

In [ ]:
time ufc_predict('Austin Hubbard','Joe Solecki', 'August 23, 2020', 'August 23, 2020')
#right winner wrong method

In [ ]:
acc_dic[13]=[1,0]

In [ ]:
time ufc_predict('Marcin Prachnio','Mike Rodriguez', 'August 23, 2020', 'August 23, 2020')
#right winner wrong decision

In [ ]:
acc_dic[14]=[1,0]

In [ ]:
time ufc_predict('Pedro Munhoz','Frankie Edgar', 'August 23, 2020', 'August 23, 2020')
#wrong winner right decision

In [ ]:
acc_dic[15]=[0,1]

In [ ]:
StipeMiocic
DanielCormier

In [ ]:
time ufc_predict('John Dodson','Merab Dvalishvili', 'August 16, 2020', 'August 16, 2020')
#right on both

In [ ]:
acc_dic[16]=[1,1]

In [ ]:
time ufc_predict('Herbert Burns','Daniel Pineda', 'August 16, 2020', 'August 16, 2020')
#right winner wrong method

In [ ]:
acc_dic[17]=[1,0]

In [ ]:
time ufc_predict('Junior Dos Santos','Jairzinho Rozenstruik', 'August 16, 2020', 'August 16, 2020')
#wrong winner right method

In [ ]:
acc_dic[18]=[0,1]

In [ ]:
time ufc_predict("Sean O'Malley",'Marlon Vera', 'August 16, 2020', 'August 16, 2020')
#wrong and wrong

In [ ]:
acc_dic[19]=[0,0]

In [ ]:
time ufc_predict("Stipe Miocic",'Daniel Cormier', 'August 16, 2020', 'August 16, 2020')
#right and right

In [ ]:
acc_dic[20]=[1,1]

In [ ]:
OmariAkhmedov
ChrisWeidman

In [ ]:
ufc_predict("Beneil Dariush",'Scott Holtzman', 'August 9, 2020', 'August 9, 2020')
#wrong and wrong

In [ ]:
acc_dic[21]=[0,0]

In [ ]:
ufc_predict("Yana Kunitskaya",'Julija Stoliarenko', 'August 9, 2020', 'August 9, 2020')

In [ ]:
acc_dic[22]=[1,1]

In [ ]:
ufc_predict("Maki Pitolo",'Darren Stewart', 'August 9, 2020', 'August 9, 2020')

In [ ]:
acc_dic[23]=[0,0]

In [ ]:
ufc_predict("Omari Akhmedov",'Chris Weidman', 'August 9, 2020', 'August 9, 2020')

In [ ]:
acc_dic[24]=[1,1]

In [ ]:
JoanneCalderwood
JenniferMaia

In [ ]:
ufc_predict("Derrick Lewis",'Aleksei Oleinik', 'August 9, 2020', 'August 9, 2020')

In [ ]:
acc_dic[25]=[1,1]

In [ ]:
ufc_predict("Lando Vannata",'Bobby Green', 'August 2, 2020', 'August 2, 2020')

In [ ]:
acc_dic[26]=[0,1]

In [ ]:
wins_before("Joanne Calderwood",'August 2, 2020')

In [ ]:
ufc_predict("Joanne Calderwood",'Jennifer Maia', 'August 2, 2020', 'August 2, 2020')

In [ ]:
acc_dic[27]=[1,0]

In [ ]:
ufc_predict("Derek Brunson",'Edmen Shahbazyan', 'August 2, 2020', 'August 2, 2020')

In [ ]:
acc_dic[28]=[0,0]

In [ ]:
ufc_predict("Khamzat Chimaev",'Rhys McKee', 'July 26, 2020', 'July 26, 2020')

In [ ]:
acc_dic[29]=[0,0]

In [ ]:
ufc_predict("Alex Oliveira",'Peter Sobotta', 'July 26, 2020', 'July 26, 2020')

In [ ]:
acc_dic[30]=[0,1]

In [ ]:
ufc_predict("Paul Craig",'Gadzhimurad Antigulov', 'July 26, 2020', 'July 26, 2020')

In [ ]:
acc_dic[31]=[0,1]

In [ ]:
ufc_predict("Carla Esparza",'Marina Rodriguez', 'July 26, 2020', 'July 26, 2020')

In [ ]:
acc_dic[32]=[1,1]

In [ ]:
ufc_predict("Fabricio Werdum",'Alexander Gustafsson', 'July 26, 2020', 'July 26, 2020')

In [ ]:
acc_dic[33]=[0,0]

In [ ]:
ufc_predict("Mauricio Rua",'Antonio Rogerio Nogueira', 'July 26, 2020', 'July 26, 2020')

In [ ]:
ufc_predict("Robert Whittaker",'Darren Till', 'July 26, 2020', 'July 26, 2020')

In [ ]:
acc_dic[34]=[1,1]

In [ ]:
ufc_predict("Alexandre Pantoja",'Askar Askarov', 'July 19, 2020', 'July 19, 2020')

In [ ]:
acc_dic[35]=[1,1]

In [ ]:
ufc_predict("Ariane Lipski",'Luana Carolina', 'July 19, 2020', 'July 19, 2020')

In [ ]:
acc_dic[36]=[0,0]

In [ ]:
ufc_predict("Marc Diakiese",'Rafael Fiziev', 'July 19, 2020', 'July 19, 2020')

In [ ]:
acc_dic[37]=[0,1]

In [ ]:
ufc_predict("Jack Hermansson",'Kelvin Gastelum', 'July 19, 2020', 'July 19, 2020')

In [ ]:
acc_dic[38]=[1,0]

In [ ]:
ufc_predict("Deiveson Figueiredo",'Joseph Benavidez', 'July 19, 2020', 'July 19, 2020')

In [ ]:
acc_dic[39]=[1,0]

In [ ]:
sum([val[0] for val in acc_dic.values()])/float(len(acc_dic.values()))

In [ ]:
sum([val[1] for val in acc_dic.values()])/float(len(acc_dic.values()))

In [ ]:
len([val for val in acc_dic.values() if val[0]==1 and val[1]==1])/float(len(acc_dic.values()))

In [ ]:
.525*.55

In [ ]:
#WHAT IN THE FUCK IS GOING ON???????

In [ ]:
list(ufc_best_df[method_mask][best_best].iloc[1000].values[3:])

In [ ]:
model.predict([list(ufc_best_df[method_mask][best_best].iloc[1000].values[3:])])

In [ ]:
[val[1] for val in acc_dic.values()]

In [ ]:
#got 12 out of 21 in both cases... not good... whats goin wrong? not enough samples?

In [ ]:
#lets check whats going on. Maybe our tuples do not match the dataframe...

In [ ]:
ufc_fights.iloc[0]

In [ ]:
ufc_best_df['fighter']=ufc_fights_clean_stance['fighter']
ufc_best_df['opponent']=ufc_fights_clean_stance['opponent']
ufc_best_df['date']=ufc_fights_clean_stance['date']

In [ ]:
test_test=['fighter','opponent','date']+best_best

In [ ]:
ufc_best_df[test_test].iloc[1000].values

In [ ]:
test_model_predict_tuple=model_prediction_tuple('Jason Knight', 'Gabriel Benitez', 'December 9, 2017','December 9, 2017')

In [ ]:
for i in range(len(test_model_predict_tuple)):
    print(test_model_predict_tuple[i],list(ufc_best_df[test_test].iloc[1000].values[3:])[i])

In [ ]:
#OK these seem right. Lets try mlp...

In [ ]:
ufc_fights_clean[mlp_test].iloc[1000].values

In [ ]:
mlp_test=mlp_prediction_tuple('Michelle Waterson','Tecia Torres','December 2, 2017','December 2, 2017')

In [ ]:
mlp_test_from_df=ufc_fights_clean[candidate].iloc[1000].values

In [ ]:
mlp_test==mlp_test_from_df

In [ ]:
for i in range(len(mlp_test)):
    print(mlp_test[i],mlp_test_from_df[i])

In [ ]:
mlp_test=['fighter','opponent','date']+candidate

In [ ]:
#Ok... is it a problem that some are ints and some are floats? Lets change everything to float
#in the dataframe, everything is a float. In the prediction tuple, many are integers... lets change this by changing
#the count function to output a float